In [1]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

### ---- Black-Scholes price for a European option ----
def bs_option_price(S, K, T, r, sigma, q=0.0, option_type="call"):
    """
    Computes the Black-Scholes price for a European call or put option.
    Parameters:
    S           : float : Spot price of the underlying asset
    K           : float : Strike price
    T           : float : Time to expiration (years)
    r           : float : Risk-free interest rate (as decimal, e.g., 0.05)
    sigma       : float : Volatility (annualized standard deviation)
    q           : float : Continuous dividend yield (as decimal, optional, default 0)
    option_type : str   : 'call' (default) or 'put'
    Returns:
    price : float : Theoretical fair value of the specified option type
    """
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return np.nan
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == "call":
        price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return price

### --- Implied volatility Function ---
def implied_volatility(market_price, S, K, T, r, q=0.0, option_type='call'):
    """
    Implied volatility using Black-Scholes, for either a call or put.
    """
    if T <= 0 or market_price <= 0 or S <= 0 or K <= 0:
        return np.nan

    def objective_function(sigma):
        return bs_option_price(S, K, T, r, sigma, q, option_type) - market_price

    try:
        implied_vol = brentq(objective_function, 1e-6, 5)
    except (ValueError, RuntimeError):
        implied_vol = np.nan
    return implied_vol


if __name__ == "__main__":
    # Test case for call option
    m = 0.5
    S = 5          # Spot price
    K = m*S        # Strike price
    T = 1.0        # Time to maturity 1 year
    r = 0.05       # Risk-free rate 5%
    q = 0.02       # Dividend yield 2%
    sigma_true = 0.321  # True volatility

    # Calculate theoretical price
    print("True Volatility: ", sigma_true )
    call_price = bs_option_price(S, K, T, r, sigma_true, q, option_type="call")
    print(f"Black-Scholes Call Price: {call_price:.10f}")

    # Now recover implied volatility from the call price
    implied_vol = implied_volatility(call_price, S, K, T, r, q, option_type="call")
    print(f"Implied Volatility from Call Price: {implied_vol:.4f}")

    # Test case for put option
    put_price = bs_option_price(S, K, T, r, sigma_true, q, option_type="put")
    print(f"Black-Scholes Put Price: {put_price:.10f}")

    implied_vol_put = implied_volatility(put_price, S, K, T, r, q, option_type="put")
    print(f"Implied Volatility from Put Price: {implied_vol_put:.4f}")

    print(f"Black-Scholes Call Price per unit spot: {call_price/S:.10f}")
    print(f"Black-Scholes Put Price per unit spot: {put_price/S:.10f}")


True Volatility:  0.321
Black-Scholes Call Price: 2.5274752258
Implied Volatility from Call Price: 0.3210
Black-Scholes Put Price: 0.0045554205
Implied Volatility from Put Price: 0.3210
Black-Scholes Call Price per unit spot: 0.5054950452
Black-Scholes Put Price per unit spot: 0.0009110841


In [2]:
import numpy as np
import scipy.stats as st

def ImpliedVolatility_vega(CP,S_0,K,sigma,tau,r):
    error    = 1e10; # initial error
    #Handy lambda expressions
    optPrice = lambda sigma: BS_Call_Option_Price(CP,S_0,K,sigma,tau,r)
    vega= lambda sigma: dV_dsigma(S_0,K,sigma,tau,r)
    
    # While the difference between the model and the arket price is large
    # follow the iteration
    n = 1.0 
    while error>10e-10:
        g         = optPrice(sigma) - V_market
        g_prim    = vega(sigma)
        sigma_new = sigma - g / g_prim
    
        #error=abs(sigma_new-sigma)
        error=abs(g)
        sigma=sigma_new;
        
        print('iteration {0} with error = {1}'.format(n,error))
        
        n= n+1
    return sigma

# Vega, dV/dsigma
def dV_dsigma(S_0,K,sigma,tau,r):
    #parameters and value of Vega
    d2   = (np.log(S_0 / float(K)) + (r - 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    value = K * np.exp(-r * tau) * st.norm.pdf(d2) * np.sqrt(tau)
    return value

def BS_Call_Option_Price(CP,S_0,K,sigma,tau,r):
    #Black-Scholes Call option price
    d1    = (np.log(S_0 / float(K)) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)
    if str(CP).lower()=="c" or str(CP).lower()=="1":
        value = st.norm.cdf(d1) * S_0 - st.norm.cdf(d2) * K * np.exp(-r * tau)
    elif str(CP).lower()=="p" or str(CP).lower()=="-1":
        value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S_0
    return value

if __name__ == "__main__":
    # Initial parameters and market quotes
    m        = 0.5    # moneyness
    S_0      = 1.0    # today's stock price
    K        = m*S_0  # strike
    tau      = 1      # time-to-maturity
    r        = 0.05   # interest rate
    sigmaInit= 0.321   # Initial implied volatility
    CP       = "c"    # C is call and P is put

    # Compute the market price from BS first (instead of hard-coding)
    V_market = BS_Call_Option_Price(CP, S_0, K, sigmaInit, tau, r)

    sigma_imp = ImpliedVolatility_vega(CP,S_0,K,sigmaInit,tau,r)
    
    message = '''Implied volatility for CallPrice= {}, strike K={}, 
      maturity T= {}, interest rate r= {} and initial stock S_0={} 
      equals to sigma_imp = {:.7f}'''.format(V_market,K,tau,r,S_0,sigma_imp)
                
    print(message)

    # Check! 
    val = BS_Call_Option_Price(CP,S_0,K,sigma_imp,tau,r)
    print('Option Price for implied volatility of {0} is equal to {1}'.format(sigma_imp, val))


iteration 1.0 with error = 0.0
Implied volatility for CallPrice= 0.5251526227243029, strike K=0.5, 
      maturity T= 1, interest rate r= 0.05 and initial stock S_0=1.0 
      equals to sigma_imp = 0.3210000
Option Price for implied volatility of 0.321 is equal to 0.5251526227243029
